In [142]:
import sqlite3

db = sqlite3.connect('../../data/school.db')
cur = db.cursor()

In [3]:
cur.execute("""
CREATE TABLE authors (
id INTEGER PRIMARY KEY AUTOINCREMENT,
wiki_id TEXT,
surname TEXT,
first_name TEXT,
second_name TEXT
birth_year INT,
birth_month INT,
birth_day INT,
death_year INT,
death_month INT,
death_day INT
)
""")
db.commit()

In [4]:
cur.execute("""
CREATE TABLE texts (
id INTEGER PRIMARY KEY AUTOINCREMENT,
link TEXT,
title TEXT,
year INT
)
""")
db.commit()

In [5]:
cur.execute("""
CREATE TABLE text_to_author (
id INTEGER PRIMARY KEY AUTOINCREMENT,
id_text INT,
id_author INT
)
""")
db.commit()

In [22]:
cur.execute("""
CREATE TABLE text_popularity (
id INTEGER PRIMARY KEY,
wiki_page INT,
pdf INT,
litres INT
)
""")
db.commit()

In [6]:
import pandas as pd

In [76]:
df = pd.read_csv('../../data/wiki_pdf_table_uni_wikidata.csv').drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1)

In [77]:
df.head()

,Litres,PDF,Wiki,Имя,Short name,Отчество,Фамилия,Название,Ссылка,Score,Авт. Права,Дата смерти,Wiki id,Авт. права wiki
0,0,0,1,NaN,NaN,NaN,Устное народное творчество,"Помню, я ещё младшенька была…",ru.wikisource.org/wiki/%D0%9F%D0%BE%D0%BC%D0%B...,1,1,NaN,Q36192,1
1,0,0,1,NaN,NaN,NaN,Устное народное творчество,"Ай вы, ветры, ветры буйные…",ru.wikisource.org/wiki/%D0%90%D0%B9_%D0%B2%D1%...,1,1,NaN,Q36192,1
2,0,0,1,Исторические,И,NaN,песни,Ермак готовится к походу на Сибирь,ru.wikisource.org/wiki/%D0%95%D1%80%D0%BC%D0%B...,1,1,NaN,Q48952271,1
3,0,0,1,Исторические,И,NaN,песни,Пугачёв в темнице,ru.wikisource.org/wiki/%D0%9F%D1%83%D0%B3%D0%B...,1,1,NaN,Q48952271,1
4,0,0,1,NaN,NaN,NaN,Русские народные сказки,Два Ивана — солдатских сына,ru.wikisource.org/wiki/%D0%9D%D0%B0%D1%80%D0%B...,1,1,NaN,Q4400636,1


In [78]:
import requests
from bs4 import BeautifulSoup

In [79]:
URL = 'https://www.wikidata.org/w/index.php'

params = {
    'sort': 'relevance',
    'search': 'Тютчев писатель',
    'profile': 'advanced',
    'fulltext': 1,
    'ns0': 1,
    'ns120': 1
}
soup = BeautifulSoup(requests.get(URL, params = params).text)

In [80]:
def get_wikiid(name):
    for profession in ['писатель','поэт','драматург', 'автор']:
        try:
            params = {
                'sort': 'relevance',
                'search': f'{name} {profession}',
                'profile': 'advanced',
                'fulltext': 1,
                'ns0': 1,
                'ns120': 1
            }
            soup = BeautifulSoup(requests.get(URL, params = params).text)
            res = soup.find_all('div', {'class': 'mw-search-result-heading'})[0].find('a')
            return res.attrs.get('href').split('/')[-1]
        except:
            pass
    raise ValueError

In [81]:
len(df[df['Wiki id'].isna()]['Фамилия'].value_counts())

37

In [71]:
wiki_additional = {}
for name in df[df['Wiki id'].isna()]['Фамилия'].value_counts().index:
    try:
        wiki_additional[name] = get_wikiid(name)
    except:
        print(name)

Пословицы и поговорки народов мира
Эпосы, легенды и сказания
Рубцов
Крупин
Пьеро
Аленушкин
Бустужев-Марлинский
Чарушин
Ишимова
Мифы древних славян


In [82]:
for key in range(df.shape[0]):
    if df.iloc[key]['Фамилия'] in wiki_additional:
        df.loc[key,'Wiki id'] = wiki_additional[df.iloc[key]['Фамилия']]

In [83]:
df[df['Wiki id'].isna()]['Фамилия'].value_counts()

Пословицы и поговорки народов мира    4
Рубцов                                3
Эпосы, легенды и сказания             3
Крупин                                3
Пьеро                                 2
Аленушкин                             1
Чарушин                               1
Бустужев-Марлинский                   1
Мифы древних славян                   1
Ишимова                               1
Name: Фамилия, dtype: int64

In [52]:
soup.find_all('div', {'class': 'mw-search-result-heading'})[0].find('a').attrs.get('href')

'/wiki/Q4467740'

In [33]:
df[df['Wiki id'].isna()]['Фамилия'].value_counts()

Тютчев                                21
Булгаков                               6
Гончаров                               5
Пословицы и поговорки народов мира     4
Носов                                  4
Твен                                   4
Эпосы, легенды и сказания              3
Рубцов                                 3
Крупин                                 3
Мольер                                 3
Тендряков                              3
Замятин                                2
Гайдар                                 2
Житков                                 2
Пьеро                                  2
Арбузов                                2
Фонвизин                               2
Байрон                                 2
Булычёв                                2
Аленушкин                              1
Бустужев-Марлинский                    1
Даррел                                 1
Чарушин                                1
Сервантес                              1
Грин            

In [92]:
authors_wiki = [tuple(row) for row in df[['Фамилия', 'Wiki id']].drop_duplicates().values]
cur.executemany('INSERT INTO authors (surname, wiki_id) VALUES (?, ?)', authors_wiki)
db.commit()

In [105]:
texts = [tuple(row) for row in df[['Название', 'Ссылка']].values]
cur.executemany('INSERT INTO texts (title, link) VALUES (?, ?)', texts)
db.commit()

In [120]:
cur.execute('SELECT surname, wiki_id, id FROM authors')
authors = {row[:2]: row[2] for row in cur.fetchall()}

In [121]:
cur.execute('SELECT title, link, id FROM texts')
texts = {row[:2]: row[2] for row in cur.fetchall()}

In [127]:
df = df.where((pd.notnull(df)), None)

In [150]:
popularity = []
authors_ = []

for key in range(df.shape[0]):
    row = df.iloc[key]
    id_text = texts[(row['Название'], row['Ссылка'])]
    id_author = authors[(row['Фамилия'], row['Wiki id'] or None)]
    popul = tuple([key+1] + list(row[['Litres', 'PDF', 'Wiki']]))
    
    popularity.append(popul)
    authors_.append((id_text, id_author))

In [131]:
cur.executemany('INSERT INTO text_to_author (id_text, id_author) VALUES (?, ?)', authors_)
db.commit()

In [145]:
list(df.iloc[0][['Litres', 'PDF', 'Wiki']])

[0, 0, 1]

In [151]:
popularity[0]

(1, 0, 0, 1)

In [153]:
import numpy as np
sqlite3.register_adapter(np.int64, lambda val: int(val))
sqlite3.register_adapter(np.int32, lambda val: int(val))

In [154]:
cur.executemany('INSERT INTO text_popularity (id, litres, pdf, wiki_page) VALUES (?, ?, ?, ?)', popularity)
db.commit()